In [29]:
import os
import shutil
import vcf
import re
import gzip
import pandas as pd
import pysam
import pybedtools
import subprocess
import numpy as np

from IPython.display import Image
from Bio import SeqIO


## import rpy2.ipython


## %load_ext rpy2.ipython

os.chdir("/master/nplatt/sch_man_nwinvasion")

# Calculate Descriptive Stats

Calculate basic descriptive stats including read count, sample coverage, probe depth etc.

In [46]:
#get merged probes
probe_bed = pybedtools.BedTool("data/renamed-sma_agilent_baits.v7.0.chr_reorderd.bed")
merged_probe_bed = probe_bed.merge(d=0)

merged_probe_bed.saveas('results/probe_coverage_and_read_counts/merged_probes.bed')


/master/nplatt/miniconda3/envs/sch_man_nwinvasion-jupyter/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


<BedTool(results/probe_coverage_and_read_counts/merged_probes.bed)>

In [106]:
#read vcf to get all samples and store based on genome or exome
vcf_reader = vcf.Reader(open('results/variant_filtration/smv7_ex_snps.vcf', 'r'))

samples={}
samples['genome'] = [x for x in vcf_reader.samples if re.search("^ER", x)]     
samples['exome'] = [x for x in vcf_reader.samples if not re.search("^ER", x)]     

#fxn to count num seqs in fastq.gz file
def num_seqs(in_fastq_gz):
    n=0
    with gzip.open(in_fastq_gz, "rt") as handle:
        for record in SeqIO.parse(handle, "fastq"):
            n=n+1
    return n

In [ ]:
#create empty df to store info
df = pd.DataFrame(index=[samples['exome'] + samples['genome']])

#get all read count info
merged_probes_bed = 'results/probe_coverage_and_read_counts/merged_probes.bed'

num_raws  = []
num_filts = []
num_maps  = []
seq_types = []
avgs_covs       = []
perc_cov_gt_10s = []
perc_cov_lt_1s  = []

#count num of raw and filtered reads
i=0
for sample in samples['exome'] + samples['genome']:
    i+=1
    print("{} of {}".format(i, len(df)))
    seq_type = ""
    
    if sample in samples['exome']:
        raw_file = "data/exomes/{}_R1.fastq.gz".format(sample)
        seq_type = "exome"
    if sample in samples['genome']:
        raw_file = "data/sra/{}_1.fastq.gz".format(sample)
        seq_type = "genome"
        
    #raw reads
    num_raw = num_seqs(raw_file)
    
    #num mapped reads
    bam_file = pysam.AlignmentFile("results/mapped_reads/{}_processed.bam".format(sample), "rb")
    num_map = bam_file.mapped

    #get coverage
    bam_file = "results/mapped_reads/{}_processed.bam".format(sample)
    out_prfx = "results/probe_coverage_and_read_counts/mosdepth/{}".format(sample)
    
    mos_cmd = "mosdepth -t 4 --by {} {} --no-per-base {}".format(merged_probes_bed, out_prfx, bam_file)
    mos_cmd=mos_cmd.split(" ")

    #run mosdepth
    process = subprocess.run(mos_cmd, 
                         stdout=subprocess.PIPE, 
                         universal_newlines=True)

    #calculate cov stats
    covs=np.array([])
    with gzip.open("results/probe_coverage_and_read_counts/mosdepth/{}.regions.bed.gz".format(sample), 'rb') as mos_infile:
        for cov_entry in mos_infile:
            chrom, start, stop, cov = cov_entry.decode("utf-8") .rstrip().split("\t") 
            cov=float(cov)
            covs=np.append(covs, cov)
        
    avg_cov = np.mean(covs)
    perc_cov_gt_10 = np.sum(covs>10)/len(covs)
    perc_cov_lt_1 = np.sum(covs<1)/len(covs)
    
    #build arrays for df import
    seq_types.append(seq_type)
    num_raws.append(num_raw)
    num_maps.append(num_map)
    avgs_covs.append(avg_cov)
    perc_cov_gt_10s.append(perc_cov_gt_10)
    perc_cov_lt_1s.append(perc_cov_lt_1)

df.insert(0,'seq_type', seq_types)
df.insert(1,'num_raw', num_raws)
df.insert(2,'mapped_reads', num_maps)
df.insert(3,'avg_probe_cov', avgs_covs)
df.insert(4,'perc_probes_gt10x', perc_cov_gt_10s)
df.insert(5,'perc_probes_failed', perc_cov_lt_1s)

csv_file = "results/probe_coverage_and_read_counts/samples_stats.csv"
df.to_csv(csv_file, index=True, header=True, mode='w')

In [140]:
df

,seq_type,num_raw,mapped_reads,avg_probe_cov,perc_probes_gt10x,perc_probes_failed
Sro_female_1.1_CCATCCTC,exome,10892425,18856533,49.880128,0.907384,0.014554
Sro_female_1.2_CCGACAAC,exome,10015111,17291766,47.860285,0.905216,0.015754
Sro_female_2.1_CCTAATCC,exome,9864727,17165447,47.281340,0.900583,0.015803
Sro_female_2.2_CCTCTATC,exome,9563172,16729520,46.810542,0.899072,0.016476
Sro_male_1.1_ATCATTCC,exome,10833101,18912188,51.479108,0.915630,0.015097
...,...,...,...,...,...,...
ERR103050,genome,92179813,180540948,44.107285,0.995745,0.001298
ERR103049,genome,124690537,241099490,64.530226,0.996337,0.001610
ERR119614,genome,124372222,244862320,60.752446,0.996090,0.001593
ERR119615,genome,118853923,232077360,59.624398,0.996386,0.001298
